In [1]:
import gc
import os
import pickle
import random
import shutil

import numpy as np
import pandas as pd
import tensorflow as tf
from keras import losses, optimizers, metrics, callbacks, Model, layers, backend as K

import SpeechModels

In [2]:
tf.config.list_logical_devices()

[LogicalDevice(name='/device:CPU:0', device_type='CPU'),
 LogicalDevice(name='/device:GPU:0', device_type='GPU')]

In [3]:
SEED = 123
N_CLASS = 12
MAX_EPOCHS = 200

In [4]:
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

## Load data

In [5]:
train_ds = tf.keras.utils.audio_dataset_from_directory(
    directory="data/train",
    batch_size=512,
    output_sequence_length=16000,
    shuffle=True,
    seed=SEED
)

val_ds = tf.keras.utils.audio_dataset_from_directory(
    directory="data/val",
    batch_size=512,
    output_sequence_length=16000,
    shuffle=False
)

test_ds = tf.keras.utils.audio_dataset_from_directory(
    directory="data/test",
    batch_size=512,
    output_sequence_length=16000,
    shuffle=False
)

label_names = np.array(train_ds.class_names)
print("label names:", label_names)

Found 45586 files belonging to 12 classes.
Found 6513 files belonging to 12 classes.
Found 13024 files belonging to 12 classes.
label names: ['down' 'go' 'left' 'no' 'off' 'on' 'right' 'silence' 'stop' 'unknown'
 'up' 'yes']


In [6]:
def squeeze(audio, labels):
    audio = tf.squeeze(audio, axis=-1)
    return audio, labels

train_ds = train_ds.map(squeeze, tf.data.AUTOTUNE)
val_ds = val_ds.map(squeeze, tf.data.AUTOTUNE)
test_ds = test_ds.map(squeeze, tf.data.AUTOTUNE)

## Model from article

In [7]:
def create_model(rate):
    m = SpeechModels.get_melspec_model(iLen=16000)
    m.trainable = False
    inputs, outputs = m.inputs, m.outputs
    x = m(inputs)
    x = tf.expand_dims(x, axis=-1, name='mel_stft')

    x = layers.Conv2D(10, (5, 1), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(1, (5, 1), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)

    # x = Reshape((125, 80)) (x)
    # keras.backend.squeeze(x, axis)
    x = layers.Lambda(lambda q: K.squeeze(q, -1), name='squeeze_last_dim')(x)

    x = layers.Bidirectional(layers.LSTM(64, return_sequences=True)
                        )(x)  # [b_s, seq_len, vec_dim]
    x = layers.Bidirectional(layers.LSTM(64, return_sequences=True)
                        )(x)  # [b_s, seq_len, vec_dim]

    x_first = layers.Lambda(lambda q: q[:, -1])(x)  # [b_s, vec_dim]
    query = layers.Dense(128)(x_first)

    # dot product attention
    att_scores = layers.Dot(axes=[1, 2])([query, x])
    att_scores = layers.Softmax(name='attSoftmax')(att_scores)  # [b_s, seq_len]

    # rescale sequence
    att_vector = layers.Dot(axes=[1, 1])([att_scores, x])  # [b_s, vec_dim]
    x = layers.Dropout(rate=rate)(att_vector)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dropout(rate=rate)(x)
    x = layers.Dense(32)(x)
    x = layers.Dropout(rate=rate)(x)
    output = layers.Dense(N_CLASS, activation='softmax', name='output')(x)

    model = Model(inputs=[inputs], outputs=[output])

    return model

model = create_model(0.1)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 16000)]      0           []                               
                                                                                                  
 normalized_spectrogram_model (  (None, 125, 80)     0           ['input[0][0]']                  
 Functional)                                                                                      
                                                                                                  
 tf.expand_dims (TFOpLambda)    (None, 125, 80, 1)   0           ['normalized_spectrogram_model[0]
                                                                 [0]']                            
                                                                                              

In [8]:
model.compile(
    optimizer=optimizers.Adam(learning_rate=0.001),
    loss=losses.SparseCategoricalCrossentropy(),
    metrics=[metrics.SparseCategoricalAccuracy(), metrics.SparseCategoricalCrossentropy()]
)

early_stopping = callbacks.EarlyStopping(
    monitor='val_sparse_categorical_accuracy',
    min_delta=0,
    patience=5,
    verbose=0,
    mode='max',
    baseline=None,
    restore_best_weights=True
)

reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_sparse_categorical_accuracy', factor=0.5, patience=3, min_lr=0.00001, verbose=1)

In [9]:
history = model.fit(
    train_ds,
    epochs=2,
    validation_data=val_ds,
    shuffle=True,
    callbacks=[early_stopping, reduce_lr]
)

Epoch 1/2
90/90 [==============================] - 30s 191ms/step - loss: 1.5906 - sparse_categorical_accuracy: 0.6232 - sparse_categorical_crossentropy: 1.5906 - val_loss: 1.5437 - val_sparse_categorical_accuracy: 0.6301 - val_sparse_categorical_crossentropy: 1.5437 - lr: 0.0010
Epoch 2/2
90/90 [==============================] - 18s 182ms/step - loss: 1.1429 - sparse_categorical_accuracy: 0.6714 - sparse_categorical_crossentropy: 1.1429 - val_loss: 1.0064 - val_sparse_categorical_accuracy: 0.6989 - val_sparse_categorical_crossentropy: 1.0064 - lr: 0.0010


## Experiments

Training will be repeated 5 times with different weights initialization.

In [10]:
TRAINING_SEEDS = list(range(5))
DROPOUTS = [0.1, 0.3, 0.5]
for dropout in DROPOUTS:
    results = []
    EXPERIMENT_NAME = f"article_net_{dropout}"
    if os.path.exists(EXPERIMENT_NAME):
        shutil.rmtree(EXPERIMENT_NAME)
        os.mkdir(EXPERIMENT_NAME)
    else:
        os.mkdir(EXPERIMENT_NAME)

    for seed in TRAINING_SEEDS:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)

        model = create_model(dropout)

        model.compile(
            optimizer=optimizers.Adam(learning_rate=0.01),
            loss=losses.SparseCategoricalCrossentropy(),
            metrics=[metrics.SparseCategoricalAccuracy(), metrics.SparseCategoricalCrossentropy()]
        )

        history = model.fit(
            train_ds,
            epochs=MAX_EPOCHS,
            validation_data=val_ds,
            shuffle=True,
            callbacks=[early_stopping, reduce_lr]
        )

        with open(os.path.join(EXPERIMENT_NAME, f"history_{seed}.pkl"), "wb") as file:
            pickle.dump(history.history, file)

        eval_results = model.evaluate(test_ds)

        predictions = model.predict(test_ds)
        with open(os.path.join(EXPERIMENT_NAME, f"predictions_{seed}.pkl"), "wb") as file:
            pickle.dump(predictions, file)

        results += [{
            'seed': seed,
            'results': dict(zip(model.metrics_names, eval_results))
        }]
        gc.collect()

    results_temp = pd.DataFrame(results)
    results_df = pd.concat([results_temp.drop(["results"], axis=1), results_temp["results"].apply(pd.Series)], axis=1)
    results_df.to_csv(os.path.join(EXPERIMENT_NAME, 'results.csv'))

Epoch 1/200
90/90 [==============================] - 24s 193ms/step - loss: 1.1920 - sparse_categorical_accuracy: 0.6640 - sparse_categorical_crossentropy: 1.1920 - val_loss: 1.6116 - val_sparse_categorical_accuracy: 0.6146 - val_sparse_categorical_crossentropy: 1.6116 - lr: 0.0100
Epoch 2/200
90/90 [==============================] - 18s 182ms/step - loss: 0.4803 - sparse_categorical_accuracy: 0.8526 - sparse_categorical_crossentropy: 0.4803 - val_loss: 1.5603 - val_sparse_categorical_accuracy: 0.6269 - val_sparse_categorical_crossentropy: 1.5603 - lr: 0.0100
Epoch 3/200
90/90 [==============================] - 17s 180ms/step - loss: 0.2730 - sparse_categorical_accuracy: 0.9176 - sparse_categorical_crossentropy: 0.2730 - val_loss: 1.4255 - val_sparse_categorical_accuracy: 0.6289 - val_sparse_categorical_crossentropy: 1.4255 - lr: 0.0100
Epoch 4/200
90/90 [==============================] - 17s 177ms/step - loss: 0.2072 - sparse_categorical_accuracy: 0.9384 - sparse_categorical_crossentr

In [11]:
results_df

,seed,loss,sparse_categorical_accuracy,sparse_categorical_crossentropy
0,0,0.198098,0.965986,0.198098
1,1,0.369765,0.963759,0.369765
2,2,0.228863,0.961763,0.228863
3,3,0.185863,0.964220,0.185863
4,4,0.297651,0.959306,0.297651
